# Lab | Agent & Vector store

**Change the state union dataset and replicate this lab by updating the prompts accordingly.**

One such dataset is the [sonnets.txt](https://github.com/martin-gorner/tensorflow-rnn-shakespeare/blob/master/shakespeare/sonnets.txt) dataset or any other data of your choice from the same git.

# Combine agents and vector stores

This notebook covers how to combine agents and vector stores. The use case for this is that you've ingested your data into a vector store and want to interact with it in an agentic manner.

The recommended method for doing so is to create a `RetrievalQA` and then use that as a tool in the overall agent. Let's take a look at doing this below. You can do this with multiple different vector DBs, and use the agent as a way to route between them. There are two different ways of doing this - you can either let the agent use the vector stores as normal tools, or you can set `return_direct=True` to really just use the agent as a router.

## Create the vector store

In [1]:
!pip install chromadb langchain langchain_community langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 75.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 69.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.6/433.6 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 68.5 MB/s eta 0:00:0

In [63]:
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

In [64]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [ ]:
# If you're using Colab, run this
import os
os.environ['OPENAI_API_KEY'] = "my key"


In [66]:
llm = OpenAI(temperature=0)

In [71]:
from pathlib import Path

relevant_parts = []
for p in Path(".").absolute().parts:
    relevant_parts.append(p)
    if relevant_parts[-3:] == ["langchain", "docs", "modules"]:
        break
doc_path = str(Path(*relevant_parts) / "1kinghenryiv.txt")

In [75]:
from google.colab import files
uploaded = files.upload()


Saving 1kinghenryiv.txt to 1kinghenryiv (1).txt


In [76]:
doc_path = "1kinghenryiv.txt"


In [80]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

# Load the document from a text file
loader = TextLoader(doc_path)
documents = loader.load()

# Split the text into chunks using a smarter splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

# Initialize OpenAI embeddings
embeddings = OpenAIEmbeddings()

# Create a Chroma vector store with persistent storage
docsearch = Chroma.from_documents(
    texts,
    embeddings,
    collection_name="state-of-union",
    persist_directory="chroma_db"  # Directory to store the Chroma database
)

# Save the vector store to disk
docsearch.persist()


<ipython-input-80-ea927092c1d5>:26: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  docsearch.persist()


In [81]:
state_of_union = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=docsearch.as_retriever()
)

In [82]:
from langchain_community.document_loaders import WebBaseLoader

In [83]:
loader = WebBaseLoader("https://beta.ruff.rs/docs/faq/")

In [85]:
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA

# Step 1: Load the document
docs = loader.load()

# Step 2: Split the document into chunks
ruff_texts = text_splitter.split_documents(docs)

# Step 3: Create Chroma vector store with a persistent directory
ruff_db = Chroma.from_documents(
    ruff_texts,
    embeddings,
    collection_name="ruff",
    persist_directory="chroma_ruff_db"  # folder to store the vector DB
)

# Optional: Persist the database to disk
ruff_db.persist()

# Step 4: Create a RetrievalQA chain using the vector store as retriever
ruff = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=ruff_db.as_retriever()
)


## Create the Agent

In [86]:
# Import things that are needed generically
from langchain.agents import AgentType, Tool, initialize_agent
from langchain_openai import OpenAI

In [87]:
tools = [
    Tool(
        name="State of Union QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
    ),
]

In [88]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [92]:
agent.invoke(
    "Why is Hotspur angry with King Henry IV?"
)



> Entering new AgentExecutor chain...
 This is a historical question, so the State of Union QA System may be more useful.
Action: State of Union QA System
Action Input: "Why is Hotspur angry with King Henry IV?"
Observation:  Hotspur is angry with King Henry IV because he believes that the king favors his son, Prince Henry, over him and does not give him the recognition and respect he deserves for his military achievements. He also resents the king's political maneuvering and sees him as a threat to his own power and influence.
Thought: This answer seems plausible, but it would be good to double check with another source.
Action: Ruff QA System
Action Input: "Why is Hotspur angry with King Henry IV?"
Observation:  I don't know.
Thought: It seems like the Ruff QA System is not able to answer historical questions. I should stick with the State of Union QA System.
Action: State of Union QA System
Action Input: "Why is Hotspur angry with King Henry IV?"
Observation:  Hotspur is angry wit

{'input': 'Why is Hotspur angry with King Henry IV?',
 'output': "Hotspur is angry with King Henry IV because he believes that the king favors his son, Prince Henry, over him and does not give him the recognition and respect he deserves for his military achievements. He also resents the king's political maneuvering and sees him as a threat to his own power and influence."}

In [93]:
agent.invoke("How does Prince Hal plan to redeem his public image?")



> Entering new AgentExecutor chain...
 This question is likely referring to a character in a literary work, so the best tool to use would be the State of Union QA System.
Action: State of Union QA System
Action Input: "How does Prince Hal plan to redeem his public image?"
Observation:  Prince Hal plans to redeem his public image by defeating Hotspur in battle and proving himself as a worthy and honorable son to his father, King Henry. He also plans to wear a garment stained with blood to symbolize his bravery and willingness to fight for his family and country.
Thought: This answer seems to be referencing Shakespeare's play, Henry IV, so it may not be entirely accurate for a real-life situation.
Action: State of Union QA System
Action Input: "Can you provide a real-life example of someone redeeming their public image?"
Observation:  One example of someone redeeming their public image is former professional cyclist Lance Armstrong. After being stripped of his seven Tour de France titl

{'input': 'How does Prince Hal plan to redeem his public image?',
 'output': 'One example of someone redeeming their public image is former professional cyclist Lance Armstrong, who admitted to his wrongdoing and dedicated himself to charity work and advocating for clean sports.'}

## Use the Agent solely as a router

You can also set `return_direct=True` if you intend to use the agent as a router and just want to directly return the result of the RetrievalQAChain.

Notice that in the above examples the agent did some extra work after querying the RetrievalQAChain. You can avoid that and just return the result directly.

In [94]:
tools = [
    Tool(
        name="State of Union QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question.",
        return_direct=True,
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
        return_direct=True,
    ),
]

In [95]:
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [97]:
agent.invoke(
    "What is the significance of the scene between Prince Hal and Falstaff at the tavern??"
)



> Entering new AgentExecutor chain...
 This question is asking for a literary analysis of a specific scene in a play.
Action: State of Union QA System
Action Input: "What is the significance of the scene between Prince Hal and Falstaff at the tavern?"
Observation:  The scene shows the relationship between Prince Hal and Falstaff, with Falstaff acting as a father figure to the prince. It also highlights Falstaff's character as a witty and humorous companion to the prince, but also reveals his flaws and weaknesses. Additionally, it foreshadows the eventual betrayal and rejection of Falstaff by Prince Hal when he becomes King Henry V.


> Finished chain.


{'input': 'What is the significance of the scene between Prince Hal and Falstaff at the tavern??',
 'output': " The scene shows the relationship between Prince Hal and Falstaff, with Falstaff acting as a father figure to the prince. It also highlights Falstaff's character as a witty and humorous companion to the prince, but also reveals his flaws and weaknesses. Additionally, it foreshadows the eventual betrayal and rejection of Falstaff by Prince Hal when he becomes King Henry V."}

In [98]:
agent.invoke("Why use ruff over flake8?")



> Entering new AgentExecutor chain...
 Ruff and flake8 are both python linters, so they have similar functions. However, there may be specific reasons why one is preferred over the other.
Action: Ruff QA System
Action Input: "Why use ruff over flake8?"
Observation:  Ruff can be used as a drop-in replacement for Flake8, but it offers a larger rule set and faster feedback on lint violations. Additionally, Ruff can also be used as a formatter, making it a more versatile tool.


> Finished chain.


{'input': 'Why use ruff over flake8?',
 'output': ' Ruff can be used as a drop-in replacement for Flake8, but it offers a larger rule set and faster feedback on lint violations. Additionally, Ruff can also be used as a formatter, making it a more versatile tool.'}

## Multi-Hop vector store reasoning

Because vector stores are easily usable as tools in agents, it is easy to use answer multi-hop questions that depend on vector stores using the existing agent framework.

In [99]:
tools = [
    Tool(
        name="State of Union QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
]

In [100]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [101]:
agent.invoke(
    "What tool does ruff use to run over Jupyter Notebooks? Did the president mention that tool in the state of the union?"
)



> Entering new AgentExecutor chain...
 I should check both tools to see if they have any information on Jupyter Notebooks.
Action: Ruff QA System
Action Input: What tool does ruff use to run over Jupyter Notebooks?
Observation:  Ruff uses Flake8 when used with any of the following plugins: colored, CLICOLOR, and CLICOLOR_FORCE.
Thought: I should check the State of Union QA System to see if the president mentioned Flake8.
Action: State of Union QA System
Action Input: Did the president mention Flake8 in the state of the union?
Observation:  I don't know.
Thought: I should check the State of Union QA System to see if the president mentioned Jupyter Notebooks.
Action: State of Union QA System
Action Input: Did the president mention Jupyter Notebooks in the state of the union?
Observation:  I don't know.
Thought: I now know the final answer.
Final Answer: The final answer is that Ruff uses Flake8 to run over Jupyter Notebooks, but it is unknown if the president mentioned it in the state 

{'input': 'What tool does ruff use to run over Jupyter Notebooks? Did the president mention that tool in the state of the union?',
 'output': 'The final answer is that Ruff uses Flake8 to run over Jupyter Notebooks, but it is unknown if the president mentioned it in the state of the union.'}